# Load Libraries

In [2]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
import warnings
warnings.filterwarnings("ignore")

## Tac information (last 3 years, commercial, closed cases, elegible architectures and elegible devices for ST)

In [3]:
df = pd.read_csv(r'C:\Users\carloma5\OneDrive - Cisco\Desktop\Work\Tac_analysis_ST\Processing\data.csv', header = 0)

In [4]:
mode_cols = ['ANALYSIS', 'PERCENTAGE', 'SEVERITY', 'COMPLEXITY', 'COMMUNICATION_CHANNEL', 'PROBLEM_CODE', 'RESOLUTION_CODE', 'REQUEST_TYPE', 'CONTRACT_STATUS', 
            'SERVICE', 'PRODUCT_ID', 'TECHNOLOGY_1', 'PRODUCT_FAMILY', 'ARCH_GROUPING', 'TIME']
d_list = mode_cols[2:14]
df_results = pd.DataFrame(df[d_list].mode(), columns = mode_cols)

mode_cols2 = ['ANALYSIS', 'PROBLEM_CODE', 'PERCENTAGE PC', 'RESOLUTION_CODE', 'PERCENTAGE RC' , 'SERVICE', 'PERCENTAGE SC', 'PRODUCT_FAMILY', 'PERCENTAGE PF']
d_list2 = [mode_cols2[i] for i in range(1,7,2)]
df_results2 = pd.DataFrame(df[d_list2].mode(), columns = mode_cols2)

## General analysis by architecture and top 3 PF.


### Returns modes and it's percentages for these columns: 
-	Severity 
-	Complexity
-	Communication_Channel
-	Problem_Code
-	Resolution_Code
-	Request_Type
-	Contract_Status
-	Service
-	Product_Id
-	Technology_1
-	Product_Family
-	Architecture


In [5]:
def mode_analysis(df_results, column, value):
   if value == 'General':
      df_results['ANALYSIS'].fillna(value, inplace=True)
      df_results['PERCENTAGE'].fillna(100, inplace=True)
      df_results['TIME'].fillna(df['TIME_TO_CLOSE_DAYS'].mean(), inplace=True)

   else: 
      df_d = df[df[column] == value]
      df_results = df_results.append(df_d[d_list].mode())
      df_results['ANALYSIS'].fillna(value, inplace=True)
      df_results['PERCENTAGE'].fillna((len(df_d)*100)/len(df), inplace=True)
      df_results['TIME'].fillna(df_d['TIME_TO_CLOSE_DAYS'].mean(), inplace=True)

   return df_results

## Top 3 deeper analysis with it's percentage representation for Architecture and PF for this columns:

-	Problem_Code
-	Resolution_Code
-	Service
-	Product_Family

In [6]:
def top3_analysis(df_results2, column, value):
    df_d = df.loc[df[column] == value]
    problem_code_counts = df_d['PROBLEM_CODE'].value_counts()
    resolution_code_counts = df_d['RESOLUTION_CODE'].value_counts()
    service_counts = df_d['SERVICE'].value_counts()
    product_family_counts = df_d['PRODUCT_FAMILY'].value_counts()

    if column == 'PRODUCT_FAMILY':
        for i in range(3):
            df_results2 = df_results2.append({
            'ANALYSIS': value, 
            'PROBLEM_CODE': problem_code_counts.index[i],
            'PERCENTAGE PC': (problem_code_counts[i] * 100) / len(df_d),
            'RESOLUTION_CODE': resolution_code_counts.index[i],
            'PERCENTAGE RC': (resolution_code_counts[i] * 100) / len(df_d),
            'SERVICE': service_counts.index[i],
            'PERCENTAGE SC': (service_counts[i] * 100) / len(df_d),
            },
            ignore_index = True
            )
    else:
        for i in range(3):
            df_results2 = df_results2.append({
            'ANALYSIS': value, 
            'PROBLEM_CODE': problem_code_counts.index[i],
            'PERCENTAGE PC': (problem_code_counts[i] * 100) / len(df_d),
            'RESOLUTION_CODE': resolution_code_counts.index[i],
            'PERCENTAGE RC': (resolution_code_counts[i] * 100) / len(df_d),
            'SERVICE': service_counts.index[i],
            'PERCENTAGE SC': (service_counts[i] * 100) / len(df_d),
            'PRODUCT_FAMILY': product_family_counts.index[i],
            'PERCENTAGE PF': (product_family_counts[i] * 100) / len(df_d)
            },
            ignore_index = True
            )
            
    return df_results2

# Results

In [8]:
mode_analysis(df_results, 'General', 'General')
l_values = df['PRODUCT_FAMILY'].value_counts().index.tolist()[:3]
l_values.extend(df['ARCH_GROUPING'].value_counts().index.tolist()[:])
#l_values.remove('Other')
for i in l_values:
    if l_values.index(i) < 3:
        df_results = mode_analysis(df_results, 'PRODUCT_FAMILY', i)
        df_results2 = top3_analysis(df_results2, 'PRODUCT_FAMILY', i)
    else:
        df_results = mode_analysis(df_results, 'ARCH_GROUPING', i)
        df_results2 = top3_analysis(df_results2, 'ARCH_GROUPING', i)
df_results2 = df_results2.iloc[1:]

with pd.ExcelWriter(r'C:\Users\carloma5\OneDrive - Cisco\Desktop\Work\Tac_analysis_ST\Results\results.xlsx') as writer:

    df_results.to_excel(writer, sheet_name = 'Mode Analysis', index = False)
    df_results2.to_excel(writer, sheet_name = 'Top3 Analysis', index = False)